In [1]:
#Imports

import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table,Column,Integer,String
import glob
import os
from sqlalchemy import MetaData
from sqlalchemy.orm import mapper

In [2]:
#SQL Alchemy setup

#Create engine that will allow us to communicate with database
engine=create_engine('sqlite:///stackedmedals.sqlite',echo=False)

#Creating session which is the middle ground to talk to our engine
Session=sessionmaker(bind=engine)
session=Session()

#Map which table in database will be related to each class
Base=declarative_base()

#Create a metadata instance
#A metadata is an object container that will store attributes and name of table 
metadata=MetaData(engine)

In [3]:
#Define structure of table
class product_table(object):
    def __init__(self,number,description,ref_des):
        self.ID=ID
        self.name_ID=name
        self.year=year
        self.gold=gold
        self.silver=silver
        self.bronze=bronze
        
    
    def __repr__(self):
        return f'{self.ID, self.name, self.year, self.gold, self.silver, self.bronze}'

In [4]:
#Declaring a table
#Defining a function that defines table, we define this function so that we can keep table names dynamic
#That is, we can have multiple tables in database and each table can have a different name
def table_definition(table_name):
    #Define table structure, here table_name varies
    #We want to make table_define available outside function so we declare it as a function attribute
    table_definition.table_define=Table(table_name,metadata,
    Column('ID',Integer,primary_key=True),
    Column('name',String),                                        
    Column('year',Integer),
    Column('gold',Integer),
    Column('silver',Integer),
    Column('bronze',Integer)
    )
    
    #Create table
    #Note that we used the engine from function
    metadata.create_all(engine)
    
    #Use mapper to define components of class as well as table definition together at once
    mapper(product_table,table_definition.table_define,non_primary=True)

In [5]:
#CREATING A DUMMY BLANK TABLE FOR PRIMARY MAPPER
#This avoids error: Class already has a primary mapper defined
#We made non_primary=True in table_definition function mapper
#This is the work around I could use, maybe there is another way

#Define table structure, here table_name varies
table_define_dummy=Table('dummy_table',metadata,
Column('ID',Integer,primary_key=True),
Column('name',String),
Column('year',Integer),
Column('gold',Integer),
Column('silver',Integer),
Column('bronze',Integer)
)

#Create table
metadata.create_all(engine)

#Use mapper to define components of class as well as table definition together at once
mapper(product_table,table_define_dummy)

<Mapper at 0x11345c0d0; product_table>

In [6]:
#This function will create a separate table for each csv file, if you have multiple csv files
#Name of table will be extracted from file name. File name contains product name. 
#Each table will be identified by product name
# It will read each excel file in the folder and insert bom into table
def create_table(file_name):
    
   #Read file into dataframe
    csv_data=pd.read_csv(file_name)

    #Convert dataframe to list and store in same variable
    csv_data=csv_data.values.tolist()

    #Get table name from file name. This will be our table name. 
    table_name_from_file='joyce'

    #Use table_definition function to define table structure
    table_definition(table_name_from_file)
    
    conn=engine.connect()

    #Loop through list of lists, each list in create_bom_table.xls_data is a row
    for row in csv_data:

        #Each element in the list is an attribute for the table class
        #Iterating through rows and inserting into table
        ins=table_definition.table_define.insert().values(
            ID=row[0],
            name=row[1],
            year=row[2],
            gold=row[3],
            silver=row[4],
            bronze=row[5])
        conn.execute(ins)

#Calling function, argument is path of folder where all CSV files are stored
create_table("joyce.csv")

In [7]:
a = pd.read_csv('joyce.csv')

In [8]:
a.ID.value_counts().max()

1

In [9]:
a = engine.execute('SELECT * FROM joyce LIMIT 10')

In [10]:
a.fetchall()

[(0, 'A North American Team', 1900, 0, 0, 4),
 (1, 'Afghanistan', 2008, 0, 0, 1),
 (2, 'Afghanistan', 2012, 0, 0, 1),
 (3, 'Algeria', 1984, 0, 0, 2),
 (4, 'Algeria', 1992, 1, 0, 1),
 (5, 'Algeria', 1996, 2, 0, 1),
 (6, 'Algeria', 2000, 1, 1, 3),
 (7, 'Algeria', 2008, 0, 1, 1),
 (8, 'Algeria', 2012, 1, 0, 0),
 (9, 'Algeria', 2016, 0, 2, 0)]